## Requirements env + moduls

In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
!cd yolov5
!pip install -r requirements.txt  # install
!pip install ultralytics
!pip install clearml
!pip install pillow==11.0.0 
!pip install requests==2.32.3 
!pip install setuptools==75.6.0 
!pip install tqdm==4.67.1
!pip install -U ipywidgets

Cloning into 'yolov5'...
remote: Enumerating objects: 17080, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 17080 (delta 21), reused 7 (delta 7), pack-reused 17050 (from 3)
Receiving objects: 100% (17080/17080), 15.68 MiB | 30.53 MiB/s, done.
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.8.1 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but 

In [2]:

import torch
from IPython import display
from IPython.display import clear_output
from pathlib import Path
import yaml
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import io
import os
import cv2
import json
import shutil
import numpy as np
from sklearn.model_selection import train_test_split

from tqdm import tqdm
%matplotlib inline

In [6]:
IMAGES_PATH = "/kaggle/input/birdies/images"
LABELS_PATH = "/kaggle/input/birdies/labels"
#NOTES_PATH = "/kaggle/input/birdies/labels"

In [7]:
labels = os.listdir(LABELS_PATH)

# Split data
train, test = train_test_split(labels, test_size=0.2, shuffle=True)
valid, test = train_test_split(test, test_size=0.8)

print(f"train: {len(train)}; test: {len(test)}; valid: {len(valid)}") #valid: {len(valid)}

train: 1608; test: 322; valid: 80


In [8]:
def move_files_to_dir(files, dirname):
    for label_filename in tqdm(files, total = len(files)):
        image_filename = f"{label_filename[:-4]}.jpg"
        shutil.copy(f"{IMAGES_PATH}/{image_filename}", f"{dirname}/images/{image_filename}")
        shutil.copy(f"{LABELS_PATH}/{label_filename}", f"{dirname}/labels/{label_filename}")


# Make folders
if not os.path.exists("test/images"):
    os.makedirs("test/images")
if not os.path.exists("test/labels"):
    os.makedirs("test/labels")
if not os.path.exists("train/images"):
    os.makedirs("train/images")
if not os.path.exists("train/labels"):
    os.makedirs("train/labels")
if not os.path.exists("valid/images"):
    os.makedirs("valid/images")
if not os.path.exists("valid/labels"):
    os.makedirs("valid/labels")

# Move splits to folders
move_files_to_dir(train, "train")
move_files_to_dir(test, "test")
move_files_to_dir(valid, "valid")

# Path
train_path = "../train/images"
test_path = "../test/images"
valid_path = "../valid/images"

100%|██████████| 80/80 [00:01<00:00, 48.30it/s]


## Make Yaml

In [9]:
with open("data.yaml", "w") as file:
    yaml.dump({
        "train": train_path,
        "test": test_path,
        "val": valid_path,
        "names": ['birds'],
        "nc": 1,
        #"names": [f'{name}' for name in names]
    } , stream=file, default_flow_style=None)

print("Now we are ready to train yolov5 model")
! ls 

Now we are ready to train yolov5 model
data.yaml  test  train	valid  yolov5


In [13]:
PROJECT_NAME = "birds"
BASE_MODEL = "yolov5m6.pt"
BATCH = 64
TRAIN_EPOCHS = 50
#VAL_BATCH = 64

In [ ]:
! python yolov5/train.py --batch $BATCH --img 640 --epochs $TRAIN_EPOCHS --data "data.yaml" --weights $BASE_MODEL --project $PROJECT_NAME --name 'feature_extraction' --cache --freeze 12

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-12-31 10:10:11.009386: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-31 10:10:11.009452: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-31 10:10:11.011151: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train:

In [ ]:
wildcard = f"{PROJECT_NAME}/validation_on_test_data*"
! rm -r $wildcard

In [ ]:
WEIGHTS_BEST = f"{PROJECT_NAME}/feature_extraction/weights/best.pt"
! python yolov5/val.py --weights $WEIGHTS_BEST --batch $VAL_BATCH --data 'data.yaml' --task test --project $PROJECT_NAME --name 'validation_on_test_data' --augment

In [ ]:
# Delete old results
wildcard = f"{PROJECT_NAME}/detect_test*"
! rm -r $wildcard

In [ ]:
! python yolov5/detect.py --weights $WEIGHTS_BEST --conf 0.6 --source 'test/images' --project $PROJECT_NAME --name 'detect_test' --augment --line=3

In [ ]:
def read_images(dirpath):
  images = []
  for img_filename in os.listdir(dirpath):
    images.append(mpimg.imread(f"{dirpath}/{img_filename}"))
  return images

def label_test_images(test_images_path, test_labels_path, classes):
  test_images = os.listdir(test_images_path)
  labeled_images = []

  for idx, test_image_filename in enumerate(test_images):
    image = mpimg.imread(f"{test_images_path}/{test_image_filename}")
    
    x_shape, y_shape = image.shape[1], image.shape[0]

    test_label_filename = f"{test_image_filename[:-4]}.txt"
    
    with open(f"{test_labels_path}/{test_label_filename}", "r") as f:
      lines = f.readlines()

      for line in lines:
        # Parse line
        box = line.split()
        class_idx = box[0]
        
        class_name = names[int(class_idx)]
        x_center, y_center, box_w, box_h = int(float(box[1])*x_shape), int(float(box[2])*y_shape), int(float(box[3])*x_shape), int(float(box[3])*y_shape)
        x1, y1, x2, y2 = x_center-int(box_w/2), y_center-int(box_h/2), x_center+int(box_w/2), y_center+int(box_h/2)

        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 3)
        cv2.putText(image, class_name, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 3)

    labeled_images.append(image)

  return labeled_images

In [ ]:
detect_path = f"{PROJECT_NAME}/detect_test"
test_images_path = f"test/images"
test_labels_path = f"test/labels"

detected_images = read_images(detect_path)
test_labeled_images = label_test_images(test_images_path, test_labels_path, classes=names)

stacked_images = [np.hstack([detected_images[idx], test_labeled_images[idx]]) for idx in range(len(detected_images))]

In [ ]:
for image in stacked_images:
  fig = plt.figure(figsize=(40, 15))
  ax1 = fig.add_subplot(2,2,1)
  ax1.imshow(image)

In [ ]:
PROJECT_FOLDER -> feature_extraction (your best) -> weights -> best.pt

In [ ]:
/kaggle/working/yolov5

In [ ]:
yolo = YOLO()

In [ ]:
model = YOLO("yolo11n-obb.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="data.yaml", epochs=2, imgsz=224)

In [ ]:
img_size = (224, 224, 3) 
bach_size = 128 
img_dir = r'/kaggle/input/birdies/images'
label_dir = r'/kaggle/input/birdies/labels'
working_dir = r'/kaggle/working/'
test_dir = r'/kaggle/input/birdies/test images'

In [ ]:
def get_box(label_dir, img_size): \
    # оптимизировано для диапазона чисел не более 2^16
    with open(label_dir) as f: # открытие файлов в ириктории
        line=f.readline()    
    box_params = line.split() # разделяем числа по пробелу
    xc = np.float16(box_params[1]) * img_size[1] # центр картинки по x
    yc = np.float16(box_params[2]) * img_size[0]   # центр картинки по y 
    
    bw = np.float16(box_params[3]) * img_size[1] # ширина
    bh = np.float16(box_params[4]) * img_size[0] # высота

    # отрисовка линий ящика
    x_min = np.int16((xc - bw/2)) # отложим от центра половину ширины вправо и влево
    
    x_max = np.int16((xc + bw/2))
    y_min = np.int16((yc - bh/2)) # отложим от центра половину длины вправо и влево
    
    y_max = np.int16((yc + bh/2))
    
    return x_min, y_min, x_max, y_max

# 

In [ ]:
get_box(label_dir + '/0010.txt', img_size)

## Отрисуем ящики на картинках

In [ ]:
# Подготовка путей и данных
img_list = np.sort(os.listdir(img_dir))
label_list = np.sort(os.listdir(label_dir))
zip_list = zip(img_list, label_list) # генератор


In [ ]:
plt.figure(figsize=(20,20))
i=0
for img, box in zip_list:
    img_path = os.path.join(img_dir, img) # соеденяем в путь к конкретной картинки
    label_path = os.path.join(label_dir, box) # получаем путь ящика
    img_draw = cv2.imread(img_path)
    img_draw = cv2.resize(img_draw, img_size[:2])
    xmin, ymin, xmax, ymax = get_box(label_path, img_size)      
        
    cv2.rectangle(img_draw, (xmin, ymin), (xmax, ymax), (255,0,0), 6) # рисуем прямоугольник на картинки
    plt.subplot(5,4, i+1)
    plt.axis('off')
    plt.imshow(img_draw)
    i += 1
 
    if i >= 20:
        break
plt.show()

In [ ]:
%time
def make_data(img_list, label_list, img_dir,label_dir, img_size):
    zip_list = zip(img_list, label_list)
    boxes = []
    images = []
    i = 1
    count = len(img_list)
    for img, label in zip_list:
        msg = f'processing image {i} of  {count}'
        i += 1
        print(msg, '\r', end='')
        img_path = os.path.join(img_dir, img)
        labelpath = os.path.join(label_dir, label)
        img_read = cv2.imread(img_path)
        img_read = cv2.resize(img_read, img_size[:2])
        xmin, ymin, xmax, ymax= get_box(label_path, img_size)  
        box = [xmin, ymin, xmax, ymax] 
        
        images.append(img_read)
        boxes.append(box)
    
    images = np.array(images)
    boxes =  np.array(boxes)
    return images, boxes

images, boxes = make_data(img_list, label_list, img_dir,label_dir, img_size)
X_train, X_test, y_train, y_test = train_test_split(images, boxes, train_size=.7, shuffle=True, random_state=142)  

## EfficientNetB1

In [ ]:
from ultralytics import YOLO
 
# Load the model.
model = YOLO('yolov8n.pt')
 
# Training.
results = model.train(
   data='custom_data.yaml',
   imgsz=640,
   epochs=10,
   batch=8,
   name='yolov8n_custom')

In [ ]:
lr = .0005

base = tf.keras.applications.efficientnet.EfficientNetB1(
    include_top=False, weights="imagenet",input_shape=img_size, pooling='avg')

base.trainable = True
x = base.output

x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x = Dense(256, kernel_regularizer = regularizers.l2(0.016), activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
x = Dropout(rate=0.33, seed=123)(x)       
output = Dense(4, activation='linear')(x)


model = Model(inputs = base.input, outputs = output)

model.compile(RMSprop(learning_rate=lr), loss='mse', metrics=['accuracy']) 